In [16]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src')

In [6]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [7]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/pricing-tool-v2/data/functions.py'>

In [8]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [9]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_otter_data.sql', 'r') as f:
    sql_template = f.read()

In [10]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_otter_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [11]:
begin_date = '2023-01-01'
end_date = '2023-02-24'
days_per_batch = 10

In [12]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)
date_list

[['2022-12-26', '2023-01-04'],
 ['2023-01-05', '2023-01-14'],
 ['2023-01-15', '2023-01-24'],
 ['2023-01-25', '2023-02-03'],
 ['2023-02-04', '2023-02-13'],
 ['2023-02-14', '2023-02-20']]

In [13]:
country_code = 'MX'
clustering_fields = ['country_code']

In [14]:
num_processes = min(cpu_count(), len(date_list))
num_processes

6

In [ ]:
res = []
with Pool(num_processes) as p:
    r = p.starmap(fu.process_otter_data_date_range, zip(repeat(sql_template), date_list, repeat(country_code), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...
Data cleaning finished!
Job 54287173-90ad-41c4-b313-75229e6e7f4b is currently in state RUNNING
Job 54287173-90ad-41c4-b313-75229e6e7f4b is currently in state RUNNING
Data cleaning finished!
Start data cleaning...
Job 4699df53-9e30-4439-9221-1c3f23f777fb is currently in state RUNNING
Job 4699df53-9e30-4439-9221-1c3f23f777fb is currently in state RUNNING
Data cleaning finished!
Job d82b8a8e-fb58-44b4-a54b-3d1ff1e29166 is currently in state RUNNING
Start data cleaning...
Start data cleaning...
Start data cleaning...
Data cleaning finished!
Job a3cfe63d-90b7-44c5-a5e9-4535f27e607a is currently in state RUNNING
Data cleaning finished!


In [17]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_otter_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [18]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [19]:
upsert_tbl = fu.upsert_tbl_otter_data(full_stag_table_name, full_prod_table_name)